# F1 All Time Races Analysis

In [216]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import re

#### Importing all the csv files

In [172]:
df = pd.read_csv('../01. Data_Gathering/Data_F1_3', index_col = [0])
df_drivers = pd.read_csv('../02. CSV_Files/drivers_info', index_col = [0])
df_circuits = pd.read_csv('../02. CSV_Files/circuits_info', index_col = [0])
df_constructors = pd.read_csv('../02. CSV_Files/constructors_info', index_col = [0])

In [173]:
df_constructors = df_constructors.transpose()
df_drivers = df_drivers.transpose()
df_circuits = df_circuits.transpose()

In [174]:
df_constructors = df_constructors[['constructorId', 'name', 'nationality']]
df_drivers = df_drivers[['driverId','givenName','familyName','dateOfBirth', 'nationality']]
df_circuits = df_circuits[['circuitId','circuitName']]

In [175]:
merge_constructors = df.merge( df_constructors, left_on = 'constructor', right_on = 'constructorId')
merge_constructors

,driver,year,round,circuit,constructor,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,constructorId,name,nationality
0,ader,1950,3,indianapolis,rae,+15 Laps,22,0.0,123,NaN,NaN,NaN,NaN,NaN,rae,Rae,American
1,agabashian,1950,3,indianapolis,kurtis_kraft,Oil leak,28,0.0,64,NaN,NaN,NaN,NaN,NaN,kurtis_kraft,Kurtis Kraft,American
2,bettenhausen,1950,3,indianapolis,kurtis_kraft,+2 Laps,5,1.0,136,NaN,NaN,NaN,NaN,NaN,kurtis_kraft,Kurtis Kraft,American
3,walt_brown,1950,3,indianapolis,kurtis_kraft,+11 Laps,19,0.0,127,NaN,NaN,NaN,NaN,NaN,kurtis_kraft,Kurtis Kraft,American
4,chitwood,1950,3,indianapolis,kurtis_kraft,+2 Laps,5,1.0,136,NaN,NaN,NaN,NaN,NaN,kurtis_kraft,Kurtis Kraft,American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24524,stroll,2019,17,suzuka,racing_point,+1 Lap,9,2.0,51,NaN,1:32.621,46.0,7.0,225.706,racing_point,Racing Point,British
24525,stroll,2019,18,rodriguez,racing_point,+1 Lap,12,0.0,70,NaN,1:20.922,68.0,15.0,191.473,racing_point,Racing Point,British
24526,stroll,2019,19,americas,racing_point,+1 Lap,13,0.0,55,NaN,1:40.380,42.0,13.0,197.716,racing_point,Racing Point,British
24527,stroll,2019,20,interlagos,racing_point,Suspension,19,0.0,65,NaN,1:12.603,65.0,10.0,213.660,racing_point,Racing Point,British


In [176]:
merge_drivers = merge_constructors.merge( df_drivers, left_on = 'driver', right_on = 'driverId')
merge_drivers

,driver,year,round,circuit,constructor,status,position,points,laps,race_time,...,fastest_lap_rank,average_speed,constructorId,name,nationality_x,driverId,givenName,familyName,dateOfBirth,nationality_y
0,ader,1950,3,indianapolis,rae,+15 Laps,22,0.0,123,NaN,...,NaN,NaN,rae,Rae,American,ader,Walt,Ader,1913-12-15,American
1,agabashian,1950,3,indianapolis,kurtis_kraft,Oil leak,28,0.0,64,NaN,...,NaN,NaN,kurtis_kraft,Kurtis Kraft,American,agabashian,Fred,Agabashian,1913-08-21,American
2,agabashian,1951,2,indianapolis,kurtis_kraft,Clutch,17,0.0,109,NaN,...,NaN,NaN,kurtis_kraft,Kurtis Kraft,American,agabashian,Fred,Agabashian,1913-08-21,American
3,agabashian,1952,2,indianapolis,kurtis_kraft,Turbo,27,0.0,71,NaN,...,NaN,NaN,kurtis_kraft,Kurtis Kraft,American,agabashian,Fred,Agabashian,1913-08-21,American
4,agabashian,1953,2,indianapolis,kurtis_kraft,Finished,4,1.5,200,+4:39.24,...,NaN,NaN,kurtis_kraft,Kurtis Kraft,American,agabashian,Fred,Agabashian,1913-08-21,American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24524,haryanto,2016,8,BAK,manor,+2 Laps,18,0.0,49,NaN,...,21.0,194.053,manor,Manor Marussia,British,haryanto,Rio,Haryanto,1993-01-22,Indonesian
24525,haryanto,2016,9,red_bull_ring,manor,+1 Lap,16,0.0,70,NaN,...,15.0,221.398,manor,Manor Marussia,British,haryanto,Rio,Haryanto,1993-01-22,Indonesian
24526,haryanto,2016,10,silverstone,manor,Spun off,19,0.0,24,NaN,...,19.0,209.189,manor,Manor Marussia,British,haryanto,Rio,Haryanto,1993-01-22,Indonesian
24527,haryanto,2016,11,hungaroring,manor,+2 Laps,21,0.0,68,NaN,...,22.0,179.649,manor,Manor Marussia,British,haryanto,Rio,Haryanto,1993-01-22,Indonesian


In [177]:
merge_circuits = merge_drivers.merge( df_circuits, left_on = 'circuit', right_on = 'circuitId')
clean_df = merge_circuits

In [178]:
clean_df['driver_name'] = clean_df['givenName'] + " " + clean_df['familyName']
clean_df = clean_df.drop(['driver','driverId', 'givenName', 'familyName','constructorId', 'circuitId', 'circuit','constructor'], axis = 1)

In [179]:
clean_df = clean_df.rename(columns = {'circuitName':'circuit_name','name':'constructor','nationality_x':'constructor_nationality','nationality_y':'drivers_nationality', 'dateOfBirth':'birth'})
clean_df


,year,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,constructor,constructor_nationality,birth,drivers_nationality,circuit_name,driver_name
0,1950,3,+15 Laps,22,0.0,123,NaN,NaN,NaN,NaN,NaN,Rae,American,1913-12-15,American,Indianapolis Motor Speedway,Walt Ader
1,1950,3,Oil leak,28,0.0,64,NaN,NaN,NaN,NaN,NaN,Kurtis Kraft,American,1913-08-21,American,Indianapolis Motor Speedway,Fred Agabashian
2,1951,2,Clutch,17,0.0,109,NaN,NaN,NaN,NaN,NaN,Kurtis Kraft,American,1913-08-21,American,Indianapolis Motor Speedway,Fred Agabashian
3,1952,2,Turbo,27,0.0,71,NaN,NaN,NaN,NaN,NaN,Kurtis Kraft,American,1913-08-21,American,Indianapolis Motor Speedway,Fred Agabashian
4,1953,2,Finished,4,1.5,200,+4:39.24,NaN,NaN,NaN,NaN,Kurtis Kraft,American,1913-08-21,American,Indianapolis Motor Speedway,Fred Agabashian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24524,2018,4,Finished,12,0.0,51,+24.720,1:48.035,38.0,15.0,200.035,Toro Rosso,Italian,1996-02-07,French,Baku City Circuit,Pierre Gasly
24525,2018,4,Finished,10,1.0,51,+18.030,1:48.288,51.0,17.0,199.567,Toro Rosso,Italian,1989-11-10,New Zealander,Baku City Circuit,Brendon Hartley
24526,2017,8,Finished,6,8.0,51,+30.298,1:45.634,39.0,9.0,204.581,Force India,Indian,1996-09-17,French,Baku City Circuit,Esteban Ocon
24527,2018,4,Collision,19,0.0,0,NaN,NaN,NaN,NaN,NaN,Force India,Indian,1996-09-17,French,Baku City Circuit,Esteban Ocon


In [181]:
df = clean_df[['year','driver_name','constructor', 'circuit_name', 'round','status', 'position', 'points', 'laps', 'race_time', 'fastest_lap_time','lap','fastest_lap_rank','average_speed', 'drivers_nationality','birth', 'constructor_nationality']]

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
0,1950,Walt Ader,Rae,Indianapolis Motor Speedway,3,+15 Laps,22,0.0,123,NaN,NaN,NaN,NaN,NaN,American,1913-12-15,American
1,1950,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,3,Oil leak,28,0.0,64,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
2,1951,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,2,Clutch,17,0.0,109,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
3,1952,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,2,Turbo,27,0.0,71,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
4,1953,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,2,Finished,4,1.5,200,+4:39.24,NaN,NaN,NaN,NaN,American,1913-08-21,American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24524,2018,Pierre Gasly,Toro Rosso,Baku City Circuit,4,Finished,12,0.0,51,+24.720,1:48.035,38.0,15.0,200.035,French,1996-02-07,Italian
24525,2018,Brendon Hartley,Toro Rosso,Baku City Circuit,4,Finished,10,1.0,51,+18.030,1:48.288,51.0,17.0,199.567,New Zealander,1989-11-10,Italian
24526,2017,Esteban Ocon,Force India,Baku City Circuit,8,Finished,6,8.0,51,+30.298,1:45.634,39.0,9.0,204.581,French,1996-09-17,Indian
24527,2018,Esteban Ocon,Force India,Baku City Circuit,4,Collision,19,0.0,0,NaN,NaN,NaN,NaN,NaN,French,1996-09-17,Indian


In [182]:
#number of points per year per constructor, ordered by points
constructor_year_points = df.groupby(['year', 'constructor']).agg({'points':'sum'})
constructor_year_points.sort_values(by = 'points', ascending = False)

,,points
year,constructor,
2016,Mercedes,765.0
2019,Mercedes,739.0
2015,Mercedes,703.0
2014,Mercedes,701.0
2017,Mercedes,668.0
...,...,...
1996,Minardi,0.0
1972,Tecno,0.0
1973,Ensign,0.0


In [183]:
df[df['year'] >= 2010]

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
1026,2011,Michael Schumacher,Mercedes,Nürburgring,10,+1 Lap,8,4.0,59,NaN,1:35.628,49.0,6.0,193.800,German,1969-01-03,German
1043,2011,Rubens Barrichello,Williams,Nürburgring,10,Oil leak,23,0.0,16,NaN,1:39.679,8.0,19.0,185.924,Brazilian,1972-05-23,British
1053,2011,Felipe Massa,Ferrari,Nürburgring,10,Finished,5,10.0,60,+52.252,1:34.609,51.0,4.0,195.888,Brazilian,1981-04-25,Italian
1054,2013,Felipe Massa,Ferrari,Nürburgring,9,Spun off,22,0.0,3,NaN,1:38.890,3.0,20.0,187.408,Brazilian,1981-04-25,Italian
1066,2013,Kimi Räikkönen,Lotus F1,Nürburgring,9,Finished,2,18.0,60,+1.008,1:33.767,57.0,2.0,197.647,Finnish,1979-10-17,British
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24524,2018,Pierre Gasly,Toro Rosso,Baku City Circuit,4,Finished,12,0.0,51,+24.720,1:48.035,38.0,15.0,200.035,French,1996-02-07,Italian
24525,2018,Brendon Hartley,Toro Rosso,Baku City Circuit,4,Finished,10,1.0,51,+18.030,1:48.288,51.0,17.0,199.567,New Zealander,1989-11-10,Italian
24526,2017,Esteban Ocon,Force India,Baku City Circuit,8,Finished,6,8.0,51,+30.298,1:45.634,39.0,9.0,204.581,French,1996-09-17,Indian
24527,2018,Esteban Ocon,Force India,Baku City Circuit,4,Collision,19,0.0,0,NaN,NaN,NaN,NaN,NaN,French,1996-09-17,Indian


##### Number of constructors

In [185]:
#number of all time constructors
len(df.circuit_name.unique())

72

#### Top 10 drivers with more point in history

In [187]:
#Top 10 drivers with more points in history
top_points_drivers = df.groupby(['driver_name']).agg({'points':'sum'})
top_points_drivers.sort_values(by='points', ascending = False).head(10)

,points
driver_name,
Lewis Hamilton,3431.0
Sebastian Vettel,2985.0
Fernando Alonso,1899.0
Kimi Räikkönen,1859.0
Nico Rosberg,1594.5
Michael Schumacher,1566.0
Valtteri Bottas,1289.0
Jenson Button,1235.0
Felipe Massa,1167.0


#### Total number of points per season

In [188]:
#total number of points per season
total_season_points = df.groupby(['year']).agg({'points':'sum'}).sort_values(by='year')
total_season_points

,points
year,
1950,167.00
1951,192.00
1952,192.00
1953,215.00
1954,215.56
...,...
2015,1919.00
2016,2121.00
2017,2020.00


#### Top 10 drivers with most races

In [202]:
top_drivers = df.groupby(['driver_name']).agg({'points':'count', 'position':'median'}).sort_values(by='points', ascending = False)
top_drivers.head(10) 

,points,position
driver_name,,
Rubens Barrichello,326,10.0
Kimi Räikkönen,315,6.0
Fernando Alonso,314,6.0
Jenson Button,309,9.0
Michael Schumacher,308,3.0
Felipe Massa,271,8.0
Riccardo Patrese,257,13.0
Jarno Trulli,256,13.0
Lewis Hamilton,250,3.0


#### Top 10 circuits

In [210]:
top_circuits = df.groupby(['year','circuit_name']).agg({'points':'count'}).sort_values(by='points', ascending = False)
top_circuits = top_circuits.groupby('circuit_name').agg({'points':'count'}).sort_values(by='points', ascending = False)
top_circuits.head(10)

,points
circuit_name,
Autodromo Nazionale di Monza,69
Circuit de Monaco,66
Silverstone Circuit,53
Circuit de Spa-Francorchamps,52
Circuit Gilles Villeneuve,40
Nürburgring,40
Hockenheimring,37
Autódromo José Carlos Pace,37
Hungaroring,34


#### Most common disqualifying reasons

In [287]:
#Finding the patter + Lap, to discard the disqualifying reasons + finished
pattern = '\+\d*\s\w*'
text = str(df['status'].unique())
result = re.findall(pattern, text) + ['Finished']
result

['+15 Laps',
 '+4 Laps',
 '+2 Laps',
 '+5 Laps',
 '+11 Laps',
 '+9 Laps',
 '+3 Laps',
 '+24 Laps',
 '+1 Lap',
 '+6 Laps',
 '+13 Laps',
 '+10 Laps',
 '+18 Laps',
 '+16 Laps',
 '+7 Laps',
 '+8 Laps',
 '+17 Laps',
 '+20 Laps',
 '+12 Laps',
 '+14 Laps',
 '+19 Laps',
 '+21 Laps',
 '+25 Laps',
 '+22 Laps',
 '+30 Laps',
 '+44 Laps',
 '+23 Laps',
 '+46 Laps',
 '+29 Laps',
 '+26 Laps',
 '+42 Laps',
 'Finished']

In [290]:
#creating a dataframe in order to discard the laps
disc_reason = df[~df['status'].isin(result)]
disc_reason

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
1,1950,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,3,Oil leak,28,0.0,64,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
2,1951,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,2,Clutch,17,0.0,109,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
3,1952,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,2,Turbo,27,0.0,71,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
6,1955,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,3,Spun off,32,0.0,39,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
8,1957,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,3,Fuel leak,22,0.0,107,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24517,2019,Daniil Kvyat,Toro Rosso,Baku City Circuit,4,Collision,19,0.0,33,NaN,1:47.681,28.0,18.0,200.692,Russian,1994-04-26,Italian
24519,2017,Max Verstappen,Red Bull,Baku City Circuit,8,Oil pressure,18,0.0,12,NaN,1:46.398,10.0,12.0,203.112,Dutch,1997-09-30,Austrian
24520,2018,Max Verstappen,Red Bull,Baku City Circuit,4,Collision,16,0.0,39,NaN,1:45.771,31.0,5.0,204.316,Dutch,1997-09-30,Austrian
24523,2019,Pierre Gasly,Red Bull,Baku City Circuit,4,Transmission,17,0.0,38,NaN,1:45.712,34.0,8.0,204.430,French,1996-02-07,Austrian


In [294]:
top_disq = disc_reason.groupby(['status']).agg({'points':'count'}).sort_values(by='points', ascending = False)
top_disq.head(10)

,points
status,
Engine,1978
Did not qualify,1024
Accident,1015
Collision,791
Gearbox,787
Spun off,785
Suspension,423
Did not prequalify,330
Transmission,318


#### Drivers who have won more races

In [296]:
position_1 = df[df.position == 1]
most_races = position_1.groupby(['driver_name']).agg({'points':'count'}).sort_values(by='points', ascending = False)
most_races.head(10)

,points
driver_name,
Michael Schumacher,91
Lewis Hamilton,84
Sebastian Vettel,53
Alain Prost,51
Ayrton Senna,41
Fernando Alonso,32
Nigel Mansell,31
Jackie Stewart,27
Jim Clark,25


#### Top 10 most 1 positions by nationality

In [299]:
position_1nat = df[df.position == 1]
most_races_nat = position_1nat.groupby(['drivers_nationality']).agg({'points':'count'}).sort_values(by='points', ascending = False)
most_races_nat.head(10)

,points
drivers_nationality,
British,289
German,179
Brazilian,101
French,79
Finnish,54
Italian,43
Australian,42
Austrian,41
Argentine,38


#### Top 10 most 1 positions by constructor

In [302]:
position_1cons = df[df.position == 1]
most_races_cons = position_1cons.groupby(['constructor']).agg({'points':'count'}).sort_values(by='points', ascending = False)
most_races_cons.head(10)

,points
constructor,
Ferrari,239
McLaren,178
Williams,114
Mercedes,102
Red Bull,62
Team Lotus,45
Renault,35
Benetton,27
Brabham,23


In [327]:
df[df.year==1950]

,year,driver_name,constructor,circuit_name,round,status,position,points,laps,race_time,fastest_lap_time,lap,fastest_lap_rank,average_speed,drivers_nationality,birth,constructor_nationality
0,1950,Walt Ader,Rae,Indianapolis Motor Speedway,3,+15 Laps,22,0.0,123,NaN,NaN,NaN,NaN,NaN,American,1913-12-15,American
1,1950,Fred Agabashian,Kurtis Kraft,Indianapolis Motor Speedway,3,Oil leak,28,0.0,64,NaN,NaN,NaN,NaN,NaN,American,1913-08-21,American
9,1950,Tony Bettenhausen,Kurtis Kraft,Indianapolis Motor Speedway,3,+2 Laps,5,1.0,136,NaN,NaN,NaN,NaN,NaN,American,1916-09-12,American
20,1950,Walt Brown,Kurtis Kraft,Indianapolis Motor Speedway,3,+11 Laps,19,0.0,127,NaN,NaN,NaN,NaN,NaN,American,1911-12-30,American
22,1950,Joie Chitwood,Kurtis Kraft,Indianapolis Motor Speedway,3,+2 Laps,5,1.0,136,NaN,NaN,NaN,NaN,NaN,American,1912-04-14,American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7242,1950,Luigi Fagioli,Alfa Romeo,Silverstone Circuit,1,Finished,2,6.0,70,+2.6,NaN,NaN,NaN,NaN,Italian,1898-06-09,Italian
7282,1950,Bob Gerard,ERA,Silverstone Circuit,1,+3 Laps,6,0.0,67,NaN,NaN,NaN,NaN,NaN,British,1914-01-19,British
7286,1950,Cuth Harrison,ERA,Silverstone Circuit,1,+3 Laps,7,0.0,67,NaN,NaN,NaN,NaN,NaN,British,1906-07-06,British
7287,1950,Leslie Johnson,ERA,Silverstone Circuit,1,Supercharger,21,0.0,2,NaN,NaN,NaN,NaN,NaN,British,1912-03-22,British
